In [3]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import gkey

In [4]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'weather_data/cities.csv'

In [2]:
city_data_df.dtypes

NameError: name 'city_data_df' is not defined

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=gkey)

In [5]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]

# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]

# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])

# Add the heatmap layer.
fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)

In [7]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [8]:
heat_layer = gmaps.heatmap_layer(locations, weights=temps)

In [9]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [10]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
6,6,Moses Lake,US,2021-08-09 19:12:46,47.1301,-119.2781,76.28,35,1,3.44
10,10,Sao Jose Da Coroa Grande,BR,2021-08-09 19:12:47,-8.8978,-35.1478,75.22,88,98,12.03
11,11,Puerto Colombia,CO,2021-08-09 19:12:47,10.9878,-74.9547,83.64,68,39,16.46
14,14,Annau,TM,2021-08-09 19:12:48,37.8876,58.5160,87.30,13,0,4.61
16,16,Thinadhoo,MV,2021-08-09 19:12:49,0.5333,72.9333,82.18,73,100,8.79
20,20,Bathsheba,BB,2021-08-09 19:12:50,13.2167,-59.5167,83.01,78,75,12.66
22,22,Cabedelo,BR,2021-08-09 19:12:51,-6.9811,-34.8339,79.47,75,27,13.51
29,29,Manado,ID,2021-08-09 19:12:53,1.4870,124.8455,75.97,88,20,2.30
31,31,Bambanglipuro,ID,2021-08-09 19:03:16,-7.9500,110.2833,75.33,90,75,6.98
35,35,Aktau,KZ,2021-08-09 19:12:54,43.6500,51.2000,86.58,34,100,3.20


In [11]:
preferred_cities_df.count()

City_ID       194
City          194
Country       194
Date          194
Lat           194
Lng           194
Max Temp      194
Humidity      194
Cloudiness    194
Wind Speed    194
dtype: int64

In [12]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
6,Moses Lake,US,76.28,47.1301,-119.2781,
10,Sao Jose Da Coroa Grande,BR,75.22,-8.8978,-35.1478,
11,Puerto Colombia,CO,83.64,10.9878,-74.9547,
14,Annau,TM,87.30,37.8876,58.5160,
16,Thinadhoo,MV,82.18,0.5333,72.9333,
20,Bathsheba,BB,83.01,13.2167,-59.5167,
22,Cabedelo,BR,79.47,-6.9811,-34.8339,
29,Manado,ID,75.97,1.4870,124.8455,
31,Bambanglipuro,ID,75.33,-7.9500,110.2833,
35,Aktau,KZ,86.58,43.6500,51.2000,


In [13]:
# Set parameters to search for a hotel.
params = {"radius": 5000, "type": "lodging", "key": gkey}

In [15]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [17]:
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
6,Moses Lake,US,76.28,47.1301,-119.2781,Ramada by Wyndham Moses Lake
10,Sao Jose Da Coroa Grande,BR,75.22,-8.8978,-35.1478,Crôa Mares Hotel
11,Puerto Colombia,CO,83.64,10.9878,-74.9547,Hotel El Emigrante
14,Annau,TM,87.30,37.8876,58.5160,Buzmein Sokak
16,Thinadhoo,MV,82.18,0.5333,72.9333,


In [18]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [19]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))